In [ ]:
!pip install requests
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
import json
from tqdm import tqdm

#Food Network. 


In [ ]:
URL = "https://www.foodnetwork.com/recipes/recipes-a-z/"

all_recipe_links = set([])
# Recipe A-Z
soup = get_soup(URL)
a_to_z_links = soup.findAll('a', attrs = {'class' : "o-IndexPagination__a-Button "})
for link in a_to_z_links:
  # link_i contains one alphabet link. 
  link_i = "https:"+ link['href']
  # Now get all pages for each alphabet. 
  print(link_i)

  count = 1
  page_link = link_i+ "/p/"+ str(count)
  r = requests.get(page_link)
  status = r.status_code
  while status!=404:
    soup = get_soup(page_link)
    recipes = soup.findAll('li', attrs = {'class' : "m-PromoList__a-ListItem"})
    for recipe in recipes:
      recipe_link = "https:"+recipe.find('a')['href']
      all_recipe_links.add(recipe_link)
    count+=1
    page_link = link_i+ "/p/"+ str(count)
    r = requests.get(page_link)
    status = r.status_code
  print(count)

https://www.foodnetwork.com/recipes/recipes-a-z/123
3
https://www.foodnetwork.com/recipes/recipes-a-z/a
18
https://www.foodnetwork.com/recipes/recipes-a-z/b
49
https://www.foodnetwork.com/recipes/recipes-a-z/c
82
https://www.foodnetwork.com/recipes/recipes-a-z/d
11
https://www.foodnetwork.com/recipes/recipes-a-z/e
10
https://www.foodnetwork.com/recipes/recipes-a-z/f
22
https://www.foodnetwork.com/recipes/recipes-a-z/g
37
https://www.foodnetwork.com/recipes/recipes-a-z/h
19
https://www.foodnetwork.com/recipes/recipes-a-z/i
8
https://www.foodnetwork.com/recipes/recipes-a-z/j
6
https://www.foodnetwork.com/recipes/recipes-a-z/k
7
https://www.foodnetwork.com/recipes/recipes-a-z/l
17
https://www.foodnetwork.com/recipes/recipes-a-z/m
31
https://www.foodnetwork.com/recipes/recipes-a-z/n
7
https://www.foodnetwork.com/recipes/recipes-a-z/o
11
https://www.foodnetwork.com/recipes/recipes-a-z/p
46
https://www.foodnetwork.com/recipes/recipes-a-z/q
4
https://www.foodnetwork.com/recipes/recipes-a-z/r


In [ ]:
all_recipe_links = list(set(all_recipe_links))

In [ ]:
import pickle
f = open('/content/drive/MyDrive/Semester 8/IR Project/FoodNetworkLinks.pickle','wb')
pickle.dump(all_recipe_links,f)
f.close()

NameError: ignored

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
import json
from tqdm import tqdm

In [ ]:
import pickle
f = open('/content/drive/MyDrive/Semester 8/IR Project/FoodNetworkLinks.pickle','rb')
all_recipe_links = pickle.load(f)
f.close()

In [ ]:
'''
Function to get the response for the specified url and parse the response using BeautifulSoup
'''
def get_soup(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    return soup

In [ ]:
len(all_recipe_links)

77502

In [ ]:
fourth = int(len(all_recipe_links)/4)

all_recipe_links = all_recipe_links[:fourth]
# all_recipe_links = all_recipe_links[fourth:2*fourth]
# all_recipe_links = all_recipe_links[2*fourth:3*fourth]
# all_recipe_links = all_recipe_links[3*fourth:]

In [ ]:
def task(all_recipe_links, val):
  all_recipes = []
  count = 0
  for recipe_link in tqdm(all_recipe_links):
    # recipe_link = "https://www.foodnetwork.com/recipes/claire-robinson/bbq-cheese-straws-recipe-1924544"
    try:
      recipe = {}
      soup = get_soup(recipe_link)
      recipe['Link'] = recipe_link
      recipe['Name'] = soup.find('span', attrs = {"class": 'o-AssetTitle__a-HeadlineText'}).text.strip()
      # Level and Total Cook Time
      one = soup.find('ul', attrs = {'class' : "o-RecipeInfo__m-Level"})
      if one is not None:
        headlines = one.findAll('span', attrs = {"class": "o-RecipeInfo__a-Headline"})
        descriptions = one.findAll('span', attrs = {"class": "o-RecipeInfo__a-Description"})
        # print(headlines)
        # print(descriptions)
        idx = 0
        for headline in headlines:
          recipe[headline.text.strip()] = descriptions[idx].text.strip()
          idx+=1

      # total = one.find('span', attrs = {"class": "o-RecipeInfo__a-Description m-RecipeInfo__a-Description--Total"})
      # recipe['Total'] = total.text.strip()

      # Prep time and Cooking Time
      two = soup.find('ul', attrs = {'class' : "o-RecipeInfo__m-Time"})
      if two is not None:
        headlines = two.findAll('span', attrs = {"class": "o-RecipeInfo__a-Headline"})
        descriptions = two.findAll('span', attrs = {"class": "o-RecipeInfo__a-Description"})
        # print(headlines)
        # print(descriptions)
        idx = 0
        for headline in headlines:
          recipe[headline.text.strip()] = descriptions[idx].text.strip()
          idx+=1

      # Yeild and Nutrition
      three = soup.find('ul', attrs = {'class' : "o-RecipeInfo__m-Yield"})
      if three is not None:
        recipe['Yield'] = three.find('span', attrs = {'class' :'o-RecipeInfo__a-Description'}).text.strip()

      nutrition = soup.find('dl',attrs = {'class':'m-NutritionTable__a-Content'})
      if nutrition is not None:
        nutrition_vals = {}
        tags = nutrition.findAll('dt', attrs = {"class": "m-NutritionTable__a-Headline"})
        values = nutrition.findAll('dd', attrs = {"class": "m-NutritionTable__a-Description"})
        idx = 0
        for tag in tags:
          tag = tag.text
          nutrition_vals[tag] = values[idx].text.strip()
          idx+=1
        recipe['Nutrition Info'] = nutrition_vals

      
      ingredients = soup.findAll('span', attrs = {'class': 'o-Ingredients__a-Ingredient--CheckboxLabel'})
      if ingredients is not None:
        ingredient_list = []
        for ingredient in ingredients:
          ingredient_list.append(ingredient.text.strip())
        recipe['Ingredients'] = ingredient_list[1:]

      
      method_list = soup.findAll('li', attrs = {'class':'o-Method__m-Step'})
      if method_list is not None:
        methods = []
        for method in method_list:
          methods.append(method.text.strip())
        recipe['Method'] = methods

      all_recipes.append(recipe)
      
    except Exception as e: 
      print(recipe_link)
      print(e)
      break;
    count+=1
    
    # Dump after every 100 recipes
    if(count%100 == 0):
      dataframe = pd.DataFrame(all_recipes)
      dataframe.to_csv('/content/drive/MyDrive/Semester 8/IR Project/_1_'+ val+ '_AllRecipes.csv')
      break
    
  dataframe = pd.DataFrame(all_recipes)
  dataframe.to_csv('/content/drive/MyDrive/Semester 8/IR Project/_1_'+ val+ '_AllRecipes.csv')
  return 

In [ ]:
from threading import Thread
fourth = int(len(all_recipe_links)/4)

t1 = Thread(target=task, args=[all_recipe_links[:fourth],"1"] )
t2 = Thread(target=task, args=[all_recipe_links[fourth: 2*fourth],"2"] )
t3 = Thread(target=task, args=[all_recipe_links[2*fourth:3*fourth],"3"] )
t4 = Thread(target=task, args=[all_recipe_links[3*fourth:],"4"] )

t1.start()
t2.start()
t3.start()
t4.start()

t1.join()
t2.join()
t3.join()
t4.join()



  3%|▎         | 504/19375 [24:28<16:06:14,  3.07s/it]



  0%|          | 0/4843 [00:00<?, ?it/s]




  0%|          | 0/4843 [00:00<?, ?it/s]






  0%|          | 0/4843 [00:00<?, ?it/s]





  0%|          | 0/4846 [00:00<?, ?it/s]



  3%|▎         | 497/19375 [24:28<12:40:10,  2.42s/it]



  0%|          | 2/4843 [00:01<1:14:55,  1.08it/s]

  3%|▎         | 504/19375 [24:31<13:36:58,  2.60s/it]


  3%|▎         | 507/19377 [24:32<18:28:02,  3.52s/it]






  0%|          | 1/4843 [00:04<5:42:29,  4.24s/it]





  0%|          | 1/4846 [00:04<5:40:58,  4.22s/it]




  3%|▎         | 498/19375 [24:33<15:42:39,  3.00s/it]



  0%|          | 3/4843 [00:05<3:04:08,  2.28s/it]


  3%|▎         | 508/19377 [24:36<19:38:58,  3.75s/it]





  0%|          | 2/4846 [00:09<6:45:48,  5.03s/it]

  3%|▎         | 499/19375 [24:38<18:19:34,  3.50s/it]



  0%|          | 4/4843 [00:10<4:07:59,  3.07s/it]






  3%|▎         | 506/19375 [24:38<21:38:05,  4.13s/it]




  0%|          | 2/484

KeyboardInterrupt: ignored



  3%|▎         | 500/19375 [24:42<19:44:25,  3.77s/it]



  0%|          | 7/4843 [00:14<2:49:03,  2.10s/it]

In [ ]:
{'Level': 'Easy',
 'Total': '30 min',
 'Prep': '15 min',
 'Cook': '15 min',
 'Yield': '6 servings',
 'Nutrition Info': {'Serving Size': '1 of 8 servings',
  'Calories': '248',
  'Total Fat': '19 g',
  'Saturated Fat': '11 g',
  'Carbohydrates': '13 g',
  'Dietary Fiber': '0 g',
  'Sugar': '0 g',
  'Protein': '8 g',
  'Cholesterol': '53 mg',
  'Sodium': '162 mg'},
 'Ingredients': ['1 1/2 cups coarsely grated extra-sharp Cheddar',
  '1 cup unbleached all-purpose flour, plus more for dusting',
  '3/4 stick cold unsalted butter, cut into tablespoons',
  '1/2 teaspoon salt',
  '1 tablespoon BBQ seasoning mix',
  '2 tablespoons heavy cream'],
 'Method': ['Preheat the oven to 350 degrees F with the racks in the upper and lower thirds of the oven. Line 2 sheet pans with parchment paper and set aside.',
  'Add the cheese, flour, butter, salt and BBQ mix to the bowl of a food processor and pulse until the mixture resembles coarse meal. Add the cream and pulse until a ball forms.',
  'On a lightly floured work surface, roll the dough out to 1/4 to 1/8-inch thick. Cut the dough with a sharp knife into 1/4-inch wide strips. Transfer to the prepared sheet pans, leaving space between them, and bake until golden brown, about 15 minutes. Flip the pans once halfway through the cooking time. Cool completely on the sheet pans.',
  'BYOC: If your friends are the spicy type, try adding a pinch or 2 of cayenne. If you want to get the kids involved, omit the BBQ spice, have them paint the straws with egg wash before you bake them and add some sesame seeds or poppy seeds.']}